In [ ]:
import pandas as pd
import numpy as np
import folium

In [ ]:
data2015 = pd.read_csv('../input/world-happiness-report-20152021/2015.csv')
data2015['Year'] = 2015 

data2016 = pd.read_csv('../input/world-happiness-report-20152021/2016.csv')
data2016['Year'] = 2016

data2017 = pd.read_csv('../input/world-happiness-report-20152021/2017.csv')
data2017['Year'] = 2017 
data2017 = data2017.rename(columns = {
    'Happiness.Rank': 'Happiness Rank', 
    'Happiness.Score': 'Happiness Score',                                   
    'Economy..GDP.per.Capita.': 'Economy (GDP per Capita)', 
    'Health..Life.Expectancy.': 'Health (Life Expectancy)',
    'Trust..Government.Corruption.': 'Trust (Government Corruption)', 
    'Dystopia.Residual': 'Dystopia Residual'
})

data2018 = pd.read_csv('../input/world-happiness-report-20152021/2018.csv')
data2018['Year'] = 2018
data2018 = data2018.rename(columns = {
    'Overall rank': 'Happiness Rank', 
    'Country or region': 'Country', 
    'Score': 'Happiness Score',                                  
    'GDP per capita': 'Economy (GDP per Capita)', 
    'Social support': 'Family',                                  
    'Healthy life expectancy': 'Health (Life Expectancy)',
    'Freedom to make life choices': 'Freedom',                                  
    'Perceptions of corruption': 'Trust (Government Corruption)'
})

data2019 = pd.read_csv('../input/world-happiness-report-20152021/2019.csv')
data2019['Year'] = 2019 
data2019 = data2019.rename(columns = {
    'Overall rank': 'Happiness Rank', 
    'Country or region': 'Country', 
    'Score': 'Happiness Score',                                 
    'GDP per capita': 'Economy (GDP per Capita)', 
    'Social support': 'Family',                                  
    'Healthy life expectancy': 'Health (Life Expectancy)',
    'Freedom to make life choices': 'Freedom', 
    'Perceptions of corruption': 'Trust (Government Corruption)'
})

data2020 = pd.read_csv('../input/world-happiness-report-20152021/2020.csv')
data2020['Happiness Rank'] =  range(1, len(data2020.index)+1)
data2020['Year'] = 2020 
data2020 = data2020.rename(columns = {
    'Country name': 'Country', 
    'Ladder score': 'Happiness Score',                                       
    'Logged GDP per capita': 'Economy (GDP per Capita)', 
    'Social support': 'Family', 
    'Healthy life expectancy': 'Health (Life Expectancy)',
    'Freedom to make life choices': 'Freedom', 
    'Perceptions of corruption': 'Trust (Government Corruption)'
})

data2021 = pd.read_csv('../input/world-happiness-report-20152021/2021.csv')
data2021['Happiness Rank'] =  range(1, len(data2021.index)+1)
data2021['Year'] = 2021
data2021 = data2021.rename(columns = {
    'Country name': 'Country', 
    'Ladder score': 'Happiness Score',           
    'Logged GDP per capita': 'Economy (GDP per Capita)', 
    'Social support': 'Family', 
    'Healthy life expectancy': 'Health (Life Expectancy)',                                  
    'Freedom to make life choices': 'Freedom', 
    'Perceptions of corruption': 'Trust (Government Corruption)'
})

cols = [
    'Country', 
    'Happiness Rank', 
    'Happiness Score', 
    'Economy (GDP per Capita)',                               
    'Family', 
    'Health (Life Expectancy)', 
    'Freedom',
    'Trust (Government Corruption)',                               
    'Generosity', 
    'Year'
]
df = pd.DataFrame(columns = cols)
n = [data2015, data2016, data2017, data2018, data2019, data2020, data2021]
for i in n:
    df = df.append(i[cols], ignore_index='True')
    
df.replace(0, np.nan, inplace=True)

df["Happiness Rank"] = df.groupby("Year")["Happiness Score"].rank("dense", ascending=False)
df["Economy Rank"] = df.groupby("Year")["Economy (GDP per Capita)"].rank("dense", ascending=False)
df["Health Rank"] = df.groupby("Year")["Health (Life Expectancy)"].rank("dense", ascending=False)

cols = [
    "Country", "Year",
    "Happiness Rank",
    "Economy Rank",
    "Health Rank",
]
df = df[cols]

df_countries = pd.read_json("https://gist.githubusercontent.com/erdem/8c7d26765831d0f9a8c62f02782ae00d/raw/248037cd701af0a4957cce340dabb0fd04e38f4c/countries.json")
df_countries.columns = ["Timezone", "LatLong", "Country", "Code", "Capital"]
df_countries[["Latitude", "Longitude"]] = df_countries["LatLong"].to_list()

In [ ]:
def get_rank_pivot(df, c):
    cols = ["2015", "2016", "2017", "2018", "2019", "2020", "2021"]

    df_pivot = df.pivot(index='Country', columns='Year', values=[c])
    df_pivot.dropna(axis = 0, inplace=True)
    df_pivot.columns = cols
    df_pivot = df_pivot.sort_values(by = df_pivot.columns[::-1].to_list())
    df_pivot.reset_index(inplace=True)

    df_pivot = pd.merge(df_pivot, df_countries, on=["Country"], how="left")
    df_pivot = df_pivot[cols + ["Country", "Latitude", "Longitude"]]
    return df_pivot.dropna()


def map_metric(df_pivot, year, c):
    source = df_pivot.sort_values(by=year).head(10)
    locations  = source[['Latitude', 'Longitude']]
    locationlist = locations.values.tolist()

    popup = source[['Country', year]]

    m = folium.Map(location=[40, 0], zoom_start=2, tiles="cartodbpositron")

    for point in range(0, len(locationlist)):
        folium.Marker(locationlist[point], popup=(f"#{popup.iloc[point,1]} {popup.iloc[point,0]}")).add_to(m)

    world_geo = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/world-countries.json'
    folium.Choropleth(
        geo_data=world_geo,
        name=f'{year} {c}',
        data=source,
        columns=['Country', year],
        key_on='feature.properties.name',
        fill_color='Blues',
        nan_fill_color='white',
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name=f'{year} {c}',
    ).add_to(m)

    title_html = '''<h3 align="Left" style="font-size:16px"><b>{}</b></h3>'''.format(f'Top 10 | {year} {c}')   
    m.get_root().html.add_child(folium.Element(title_html))
    return m


def show_top10(df_pivot, c):
    cols = ["Country", "2015", "2016", "2017", "2018", "2019", "2020", "2021"]

    return (df_pivot[cols].head(10).set_index("Country")
    .style
    .format('{:.0f}')
    .background_gradient(cmap='Greys')
    .highlight_min(color='lightgreen')
    .highlight_max(color='#cd4f39')
    .set_caption(f"Top 10 | {c}")
    )

In [ ]:
c = "Happiness Rank"
year = "2021"

df_pivot = get_rank_pivot(df=df, c=c)
show_top10(df_pivot, c)

In [ ]:
map_metric(df_pivot=df_pivot, c=c, year=year)

In [ ]:
c = "Economy Rank"
year = "2021"

df_pivot = get_rank_pivot(df=df, c=c)
show_top10(df_pivot, c)

In [ ]:
map_metric(df_pivot=df_pivot, c=c, year=year)

In [ ]:
c = "Health Rank"
year = "2021"

df_pivot = get_rank_pivot(df=df, c=c)
show_top10(df_pivot, c)

In [ ]:
map_metric(df_pivot=df_pivot, c=c, year=year)